In [1]:
from scipy.special import gammaln, betaln, psi
from scipy.optimize import check_grad
from scipy.optimize import minimize, approx_fprime
from math_special_functions import multibetaln, dibetaln, dimultibetaln
import numpy as np

In [2]:
class ShmKmerLikelihoodOLD:
    def __init__(self, sample, nonmutated_ind, check_gradient=False, number_of_tests=None):
        self.full_sample = sample
        self.mutated_sample = np.delete(sample, nonmutated_ind, 1)
        self.nonmutated_ind = nonmutated_ind

        self.n_all = np.sum(self.full_sample, axis=1)
        self.n_mutated = np.sum(self.mutated_sample, axis=1)

        if check_gradient:
            if number_of_tests is None:
                print("Supply number of tests for check_gradient.")
            else:
                self.__check_gradient(number_of_tests)

    def likelihood(self, beta_shape1, beta_shape2, dir_lambda):
        result_p11 = betaln(beta_shape1 + self.n_mutated,
                            beta_shape2 + self.n_all - self.n_mutated)
        result_p12 = betaln(beta_shape1, beta_shape2)
        
        result_p21 = multibetaln(self.mutated_sample + dir_lambda, 1)
        result_p22 = multibetaln(dir_lambda)
            
        return np.sum(result_p11 - result_p12 + result_p21 - result_p22)

    def gradient(self, beta_shape1, beta_shape2, dir_lambda):
        N = self.full_sample.shape[0]
        result_p11 = np.sum(dibetaln(beta_shape1 + self.n_mutated,
                           beta_shape2 + self.n_all - self.n_mutated), 0)
        result_p12 = dibetaln(beta_shape1, beta_shape2).flatten() * N

        result_p21 = np.sum(dimultibetaln(self.mutated_sample + dir_lambda), 0)
        result_p22 = dimultibetaln(dir_lambda) * N

        return np.concatenate((result_p11 - result_p12,
                               result_p21 - result_p22))

    def __check_gradient(self, number_of_tests=None):
        cglikelihood = (lambda x: self.likelihood(beta_shape1=x[0],
                                                  beta_shape2=x[1],
                                                  dir_lambda=x[2:]))
        cggrad = (lambda x: self.gradient(beta_shape1=x[0],
                                          beta_shape2=x[1],
                                          dir_lambda=x[2:]))
        if number_of_tests is not None:
            for i in xrange(number_of_tests):
                point = np.random.exponential(size=5, scale=10)
                print(check_grad(cglikelihood, cggrad, x0=point))
        if x is not None:
            print(check_grad(cglikelihood, cggrad, x))

In [3]:
class ShmKmerLikelihoodOptimizatorOLD:
    def __init__(self, shm_kmer_likelihood, x0=None, bounds=None, method='L-BFGS-B'):
        if x0 is None:
            self.x0 = self.__get_start_point(shm_kmer_likelihood)
        if bounds is None:
            self.bounds=((0, None),) * (2 + shm_kmer_likelihood.mutated_sample.shape[1])
        self.method=method
        self.lkhd = (lambda x: -shm_kmer_likelihood.likelihood(beta_shape1=x[0],
                                                               beta_shape2=x[1],
                                                               dir_lambda=x[2:]))
        self.grad = (lambda x: -shm_kmer_likelihood.gradient(beta_shape1=x[0],
                                                             beta_shape2=x[1],
                                                             dir_lambda=x[2:]))
        
    def __get_start_point(self, shm_kmer_likelihood, scale_beta=1, scale_dir=1):
        beta_shape1 = np.mean(1 - shm_kmer_likelihood.full_sample[:,shm_kmer_likelihood.nonmutated_ind] / \
                              np.sum(shm_kmer_likelihood.full_sample, axis=1, dtype=float))
        beta_shape2 = 1 - beta_shape1
        beta_shape1 *= scale_beta
        beta_shape2 *= scale_beta
        
        scaled_mutated_sample = (shm_kmer_likelihood.mutated_sample.T / \
                                 np.sum(shm_kmer_likelihood.mutated_sample, axis=1, dtype=float)).T
        dir_lambda = np.mean(scaled_mutated_sample, axis=0, dtype=float)
        dir_lambda *= scale_dir
        return np.concatenate([np.array([beta_shape1, beta_shape2]), dir_lambda])
    
    def maximize(self):
        minimize_result = minimize(fun=self.lkhd,
                                   x0=self.x0,
                                   bounds=self.bounds,
                                   method=self.method,
                                   jac=self.grad)
        return minimize_result

In [4]:
from shm_kmer_likelihood_optimize import ShmKmerLikelihood, ShmKmerLikelihoodOptimizator
from generate_test_sample import generate_sample

In [57]:
sample = generate_sample(sample_size=1000,
                         number_samples=10,
                         real_beta_shape1=5,
                         real_beta_shape2=15,
                         real_dir_lambda=[4,5,6],
                         nonmutated_ind=0)

In [58]:
sample

array([[825,  43,  88,  44],
       [825,  66,  38,  71],
       [681, 114,  85, 120],
       [758,  46, 124,  72],
       [767,  75,  99,  59],
       [856,  28,  86,  30],
       [775,  83,  36, 106],
       [665,  28, 193, 114],
       [736,  49, 103, 112],
       [626, 100, 124, 150]])

In [59]:
lkho = ShmKmerLikelihood(sample, 0)
lkho_old = ShmKmerLikelihoodOLD(sample, 0)

In [60]:
lkhoo = ShmKmerLikelihoodOptimizator(lkho)
lkhoo_old = ShmKmerLikelihoodOptimizatorOLD(lkho_old)

In [61]:
lkhoo.maximize()

(      fun: 5492.5847881541868
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.85342285e-06,   6.81973513e-07])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 17
      nit: 16
   status: 0
  success: True
        x: array([  9.3882882 ,  28.37167499]),
       fun: 2594.8816967749867
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -8.17272631e-06,  -1.57742297e-05,   4.02029054e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 15
      nit: 14
   status: 0
  success: True
        x: array([ 4.68960327,  6.96849514,  6.34373814]))

In [62]:
lkhoo_old.maximize()

      fun: 8087.4664860037483
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -6.43493214e-04,   2.92009344e-04,  -4.78013947e-06,
         3.24464620e-04,  -4.54507542e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 21
   status: 0
  success: True
        x: array([  9.39242926,  28.38732679,   4.68924881,   6.96818506,   6.34292881])